<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/gpt_from_scracth/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação de bibliotecas

In [1]:
!pip install torch
!pip install typing
!pip install pydantic

Primeira versão, básica. Sem córtex pré-frontal

In [ ]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List
from pydantic import BaseModel

class GPTInput(BaseModel):
    text: str

class GPT:
    # Mover os métodos para dentro da classe GPT com a indentação correta
    def init(self, caracteres: str):
        self.raw_text = caracteres
        self.tokens = self.tokenize(caracteres)
        self.vocab = list(set(self.tokens))
        self.token_to_idx = {ch: i for i, ch in enumerate(self.vocab)}
        self.idx_to_token = {i: ch for ch, i in self.token_to_idx.items()}
        self.encoded = self.encode(self.tokens)
        self.embedding_dim = 16
        self.embeddings = self.init_embeddings(len(self.vocab), self.embedding_dim)
        self.weights_q = self.init_weights()
        self.weights_k = self.init_weights()
        self.weights_v = self.init_weights()
        self.positional_encoding = self.get_positional_encoding(len(self.encoded), self.embedding_dim)
        self.ffn1 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)
        self.ffn2 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)

    def tokenize(self, text: str) -> List[str]:
        return list(text)

    def encode(self, tokens: List[str]) -> List[int]:
        return [self.token_to_idx[tok] for tok in tokens]

    def init_embeddings(self, vocab_size: int, dim: int):
        return torch.randn(vocab_size, dim, requires_grad=True)

    def init_weights(self):
        return torch.randn(self.embedding_dim, self.embedding_dim, requires_grad=True)

    def get_positional_encoding(self, seq_len: int, dim: int):
        pe = torch.zeros(seq_len, dim)
        for pos in range(seq_len):
            for i in range(0, dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/dim)))
                if i + 1 < dim:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i)/dim)))
        return pe

    def embed(self, encoded: List[int]):
        x = torch.stack([self.embeddings[idx] for idx in encoded])
        x += self.positional_encoding
        return x

    def attention(self, x: torch.Tensor):
        Q = x @ self.weights_q
        K = x @ self.weights_k
        V = x @ self.weights_v

        dk = Q.size(-1)
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(dk)
        weights = F.softmax(scores, dim=-1)
        attended = weights @ V

        # Cálculo simbólico do número de arranjos com repetição
        n = Q.shape[0]
        num_arranjos = n ** 2
        print(f"Número de arranjos com repetição em atenção: {num_arranjos}")

        # Cálculo de combinações ponderadas entre posições Q e V
        for i in range(n):
            for j in range(n):
                prob = weights[i, j].item()
                print(f"Probabilidade de atenção do token {i} para {j}: {prob:.4f}")

        return attended

    def feed_forward(self, x):
        x = F.relu(self.ffn1(x))
        return self.ffn2(x)

    def forward(self):
        x = self.embed(self.encoded)
        x = self.attention(x)
        x = self.feed_forward(x)
        return x

    def train_step(self, target: torch.Tensor, optimizer):
        output = self.forward()
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    def validate(self):
        # Validação Bayesiana (simulação simples)
        print("Validação Bayesiana placeholder: usando heurísticas de incerteza para ajustar hiperparâmetros")
        return {
            "melhor_lr": 0.001,
            "melhor_batch_size": 8,
            "incerteza_modelo": 0.12
        }

    def retrain(self, target: torch.Tensor):
        print("Reiniciando com hiperparâmetros otimizados")
        opt = torch.optim.Adam([self.embeddings], lr=0.001)
        for i in range(10):
            loss = self.train_step(target, opt)
            print(f"Iteração {i+1} - Loss: {loss:.4f}")

# Exemplo de uso:

entrada = GPTInput(text="transformer")
# Note: The __init__ method should be defined as __init__ not init
modelo = GPT()
modelo.init(entrada.text)
target = torch.randn(len(modelo.encoded), modelo.embedding_dim)
modelo.retrain(target)
saida = modelo.forward()
print(saida)

Reiniciando com hiperparâmetros otimizados
Número de arranjos com repetição em atenção: 121
Probabilidade de atenção do token 0 para 0: 0.0000
Probabilidade de atenção do token 0 para 1: 0.0000
Probabilidade de atenção do token 0 para 2: 0.0000
Probabilidade de atenção do token 0 para 3: 0.0000
Probabilidade de atenção do token 0 para 4: 0.0000
Probabilidade de atenção do token 0 para 5: 0.0000
Probabilidade de atenção do token 0 para 6: 0.0000
Probabilidade de atenção do token 0 para 7: 0.0000
Probabilidade de atenção do token 0 para 8: 0.0000
Probabilidade de atenção do token 0 para 9: 1.0000
Probabilidade de atenção do token 0 para 10: 0.0000
Probabilidade de atenção do token 1 para 0: 0.0000
Probabilidade de atenção do token 1 para 1: 0.0000
Probabilidade de atenção do token 1 para 2: 0.0000
Probabilidade de atenção do token 1 para 3: 1.0000
Probabilidade de atenção do token 1 para 4: 0.0000
Probabilidade de atenção do token 1 para 5: 0.0000
Probabilidade de atenção do token 1 para

Segunda versão, já com a arquitetura base token + embedding - posicional enconding - self attention - attention - multi head attention (mha) - ferro forward neural netowork (ffn) - resíduos e camadas de normalização. Inclui a tomada de decisão via córtex pré-frontal

In [3]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List
from pydantic import BaseModel

class GPTInput(BaseModel):
    text: str

class MiniPrefrontalCortex:
    # Adicionar um linear layer para mapear de embedding_dim para o número de ações
    def __init__(self, actions, embedding_dim, temperature=1.0, top_k=None):
        self.actions = actions
        self.num_actions = len(actions)
        self.temperature = temperature
        self.top_k = top_k
        # Linear layer para mapear a saída do modelo para o espaço de ações
        self.decision_layer = nn.Linear(embedding_dim, self.num_actions)

    def softmax(self, logits):
        # Aplicar a camada linear antes do softmax
        action_logits = self.decision_layer(torch.from_numpy(logits)).detach().numpy()
        scaled_logits = action_logits / self.temperature
        exps = np.exp(scaled_logits - np.max(scaled_logits))
        return exps / np.sum(exps)

    def apply_top_k(self, probs):
        if self.top_k is None or self.top_k >= len(probs):
            return probs
        # Certificar-se de que top_k não excede o número real de ações
        actual_top_k = min(self.top_k, len(probs))
        indices = np.argpartition(probs, -actual_top_k)[-actual_top_k:]
        mask = np.zeros_like(probs)
        mask[indices] = 1
        filtered = probs * mask
        # Normalizar as probabilidades filtradas
        return filtered / np.sum(filtered) if np.sum(filtered) > 0 else np.ones_like(probs) / len(probs)


    def decide(self, logits):
        probs = self.softmax(logits)
        probs = self.apply_top_k(probs)
        # np.random.choice requer que a soma das probabilidades seja 1 (ou muito próxima)
        # e que o tamanho de p seja igual ao de a.
        # Garantir que a soma seja 1 para evitar erros
        probs = probs / np.sum(probs) if np.sum(probs) > 0 else np.ones_like(probs) / len(probs)
        action_idx = np.random.choice(self.num_actions, p=probs)
        return self.actions[action_idx], probs

class GPT:
    def __init__(self, caracteres: str):
        self.raw_text = caracteres
        self.tokens = self.tokenize(caracteres)
        self.vocab = list(set(self.tokens))
        self.token_to_idx = {ch: i for i, ch in enumerate(self.vocab)}
        self.idx_to_token = {i: ch for ch, i in self.token_to_idx.items()}
        self.encoded = self.encode(self.tokens)
        self.embedding_dim = 16
        self.embeddings = self.init_embeddings(len(self.vocab), self.embedding_dim)
        self.weights_q = self.init_weights()
        self.weights_k = self.init_weights()
        self.weights_v = self.init_weights()
        self.positional_encoding = self.get_positional_encoding(len(self.encoded), self.embedding_dim)
        self.ffn1 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)
        self.ffn2 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)

        # Mini-córtex com ações possíveis, passando o embedding_dim
        self.actions = ["Gerar Próximo Token", "Revisar Saída", "Aumentar Atenção", "Reduzir Atenção", "Finalizar"]
        self.mini_cortex = MiniPrefrontalCortex(self.actions, self.embedding_dim, temperature=1.0, top_k=3)


    def tokenize(self, text: str) -> List[str]:
        return list(text)

    def encode(self, tokens: List[str]) -> List[int]:
        return [self.token_to_idx[tok] for tok in tokens]

    def init_embeddings(self, vocab_size: int, dim: int):
        return torch.randn(vocab_size, dim, requires_grad=True)

    def init_weights(self):
        return torch.randn(self.embedding_dim, self.embedding_dim, requires_grad=True)

    def get_positional_encoding(self, seq_len: int, dim: int):
        pe = torch.zeros(seq_len, dim)
        for pos in range(seq_len):
            for i in range(0, dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/dim)))
                if i + 1 < dim:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i)/dim)))
        return pe

    def embed(self, encoded: List[int]):
        x = torch.stack([self.embeddings[idx] for idx in encoded])
        x += self.positional_encoding
        return x

    def attention(self, x: torch.Tensor):
        Q = x @ self.weights_q
        K = x @ self.weights_k
        V = x @ self.weights_v

        dk = Q.size(-1)
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(dk)
        weights = F.softmax(scores, dim=-1)
        attended = weights @ V

        n = Q.shape[0]
        num_arranjos = n ** 2
        print(f"Número de arranjos com repetição em atenção: {num_arranjos}")
        for i in range(n):
            for j in range(n):
                prob = weights[i, j].item()
                print(f"Probabilidade de atenção do token {i} para {j}: {prob:.4f}")

        return attended

    def feed_forward(self, x):
        x = F.relu(self.ffn1(x))
        return self.ffn2(x)

    def forward(self):
        x = self.embed(self.encoded)
        x = self.attention(x)
        x = self.feed_forward(x)
        return x

    def decide_with_prefrontal(self):
        output = self.forward()
        # Média sobre a dimensão da sequência para obter um tensor [embedding_dim]
        logits_for_decision = output.mean(dim=0).detach().numpy()
        # Passar os logits para o mini-córtex para decidir a ação
        action, probs = self.mini_cortex.decide(logits_for_decision)
        print(f"Ação decidida pelo MiniCórtex: {action}")
        print(f"Distribuição de probabilidades: {np.round(probs,3)}")
        return action, probs


    def train_step(self, target: torch.Tensor, optimizer):
        output = self.forward()
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    def retrain(self, target: torch.Tensor):
        print("Reiniciando com hiperparâmetros otimizados")
        opt = torch.optim.Adam([self.embeddings, *self.mini_cortex.decision_layer.parameters()], lr=0.001) # Incluir os parâmetros da camada de decisão
        for i in range(10):
            loss = self.train_step(target, opt)
            print(f"Iteração {i+1} - Loss: {loss:.4f}")

# Uso exemplo
entrada = GPTInput(text="transformer")
modelo = GPT(entrada.text)
target = torch.randn(len(modelo.encoded), modelo.embedding_dim)
modelo.retrain(target)

# Decisão do mini-córtex integrada
action, probs = modelo.decide_with_prefrontal()

Reiniciando com hiperparâmetros otimizados
Número de arranjos com repetição em atenção: 121
Probabilidade de atenção do token 0 para 0: 0.0000
Probabilidade de atenção do token 0 para 1: 0.0000
Probabilidade de atenção do token 0 para 2: 0.0000
Probabilidade de atenção do token 0 para 3: 0.0000
Probabilidade de atenção do token 0 para 4: 0.0000
Probabilidade de atenção do token 0 para 5: 1.0000
Probabilidade de atenção do token 0 para 6: 0.0000
Probabilidade de atenção do token 0 para 7: 0.0000
Probabilidade de atenção do token 0 para 8: 0.0000
Probabilidade de atenção do token 0 para 9: 0.0000
Probabilidade de atenção do token 0 para 10: 0.0000
Probabilidade de atenção do token 1 para 0: 0.0000
Probabilidade de atenção do token 1 para 1: 0.0000
Probabilidade de atenção do token 1 para 2: 0.0000
Probabilidade de atenção do token 1 para 3: 0.0000
Probabilidade de atenção do token 1 para 4: 0.0000
Probabilidade de atenção do token 1 para 5: 1.0000
Probabilidade de atenção do token 1 para